In [1]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import datetime
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait, Select
from webdriver_manager.chrome import ChromeDriverManager


service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)



data_de_hoje = datetime.date.today()

# Coleta os produtos da loja
produtos = []

# === CONFIGURAÇÃO INICIAL ===
# data_de_hoje = datetime.date.today()
# produtos = []

options = webdriver.ChromeOptions()
# options.add_argument("--headless=new")  # comente para visualizar
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://mercado.carrefour.com.br/")
time.sleep(3)


# Busca pelo nome do producto
search = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Pesquise por produtos ou marcas']"))
)
search.click()
search.send_keys("azeite")
search.send_keys(Keys.RETURN)

In [2]:
search = WebDriverWait(driver, 1).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='open-modal']"))
)
search.click()

# Reire na loja
retire = WebDriverWait(driver, 1).until(
    EC.element_to_be_clickable((By.XPATH, "//button[span[contains(text(), 'Retire na loja')]]"))
)
retire.click()

# Seleciona a cidade (exemplo: primeira cidade válida)
select = Select(WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "select.w-full.border-gray-300.rounded.p-3.mt-4.cursor-pointer.text-sm.drop-shadow-md"))
))

total_cidades = len(select.options) - 1  # Subtrai 1 se quiser ignorar a primeira opção ("Selecionar cidade")
print(f"[INFO] Total de cidades disponíveis: {total_cidades}")



[INFO] Total de cidades disponíveis: 56


In [7]:
for i in range(41, 55):
    #### 2 - MANAUS ####
    select.select_by_index(i)  # Exemplo: primeira cidade após o "Selecionar cidade"

    # Aguarda até que as lojas da cidade carreguem no DOM
    loja_elements = WebDriverWait(driver, 2).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.border.rounded-lg.shadow-md.p-4"))
    )
    # Conta e informa o total de lojas
    total_lojas = len(loja_elements)
    print(f"[INFO] Total de lojas na cidade {i}: {total_lojas}")



    for loja_index in range(len(loja_elements)):
        # Recarrega a lista de lojas antes de cada clique
        lojas_atualizadas = driver.find_elements(By.CSS_SELECTOR, "li.border.rounded-lg.shadow-md.p-4")
        time.sleep(1)
        loja = lojas_atualizadas[loja_index]
        nome_loja = loja.text.strip()
        print(f"[Cidade {i}] Loja selecionada: {nome_loja}")
        loja.click()
        time.sleep(3)


        # Scroll para carregar todos os produtos
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)

        # Captura os blocos de produto
        divs_produtos = driver.find_elements(By.CSS_SELECTOR, "a[data-testid='search-product-card']")

        print(f"[INFO] Total de blocos de produto encontrados: {len(divs_produtos)}")

        for idx in range(len(divs_produtos)):
            try:
                # Recarrega a lista para evitar StaleElementReferenceException
                divs_produtos_atualizados = driver.find_elements(By.CSS_SELECTOR, "a[data-testid='search-product-card']")
                div = divs_produtos_atualizados[idx]

                texto_produto = div.text.strip()

                if texto_produto:
                    # Formata igual ao outro código: produto + loja + cidade
                    registro = f"{texto_produto}\n{nome_loja}"
                    produtos.append(registro)

            except Exception as e:
                print(f"[WARNING] Erro ao coletar produto {idx+1}: {e}")
                continue

        print(f"[INFO] Total de produtos coletados até agora: {len(produtos)}")



        # Antes de voltar para a próxima loja:
        driver.back()
        time.sleep(3)

        # Agora, reabrir o picker de cidade
        # Faz um clique genérico no body para destravar o picker
        WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div[class*='container-default']"))
        ).click()
        time.sleep(1)
        
        # Busca por cidade
        search = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='open-modal']"))
        )
        search.click()
        
        WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//button[span[contains(text(), 'Retire na loja')]]"))
        ).click()

        # E aqui novamente... recaptura o Select ANTES de re-selecionar a cidade!
        select = Select(WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "select.w-full.border-gray-300.rounded.p-3.mt-4.cursor-pointer.text-sm.drop-shadow-md"))
        ))
        #select.select_by_index(cidade_index)

        
        select.select_by_index(i)

        time.sleep(2)


driver.quit()

    

[INFO] Total de lojas na cidade 41: 1
[Cidade 41] Loja selecionada: Hiper Nova Piracicaba
Avenida Ruy Teixeira Mendes
Glebas Califórnia, Piracicaba - SP
[INFO] Total de blocos de produto encontrados: 18
[INFO] Total de produtos coletados até agora: 1396
[INFO] Total de lojas na cidade 42: 1
[Cidade 42] Loja selecionada: Bairro Praia Grande
Avenida Castelo Branco
Vila Guilhermina, Praia Grande - SP
[INFO] Total de blocos de produto encontrados: 17
[INFO] Total de produtos coletados até agora: 1413
[INFO] Total de lojas na cidade 43: 1
[Cidade 43] Loja selecionada: Hiper Prudente
Avenida Manoel Goulart
Jardim das Rosas, Presidente Prudente - SP
[INFO] Total de blocos de produto encontrados: 18
[INFO] Total de produtos coletados até agora: 1431
[INFO] Total de lojas na cidade 44: 3
[Cidade 44] Loja selecionada: Hiper Ribeirão Shopping
Avenida Coronel Fernando Ferreira Leite
Jardim California, Ribeirão Preto - SP
[INFO] Total de blocos de produto encontrados: 18
[INFO] Total de produtos co

In [8]:
i

54

In [9]:
produtos

['Patrocinado\nTempero Granulado para Frango Maggi Meu Segredo Pacote 50g 10 Unidades de 5g Cada\nR$ 4,99\n-34%\nR$ 3,29\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Azeite Português Extra Virgem Andorinha Dorinha 500ml\nR$ 34,69\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Azeite Português Extra Virgem Tradicional Gallo 500ml\nR$ 38,89\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Azeite Terras de Camões Extra Virgem Frutado 500ml\nR$ 33,99\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Azeite Extra Virgem Allegro 500ml\nR$ 36,99\n-10%\nR$ 33,29\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Kit Azeite Terras de Camões Extra Virgem Frutado 500ml 3 Unidades\nR$ 101,97\nADICIONAR\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\n

In [10]:
len(produtos)

2158

In [11]:
produtos_limpos = []

for p in produtos:
    # Remover linha se for "Produto indisponível" ou "lencol"
    if 'Produto indisponível' in p or 'lencol' in p.lower():
        continue

    # Remover a palavra "ADICIONAR"
    p = p.replace('ADICIONAR', '').strip()

    # Remover qualquer preço no formato por kg, litro, etc
    p = re.sub(r"R\$ ?\d+,\d{2}/(kg|g|l|ml|100ml|un|litro)", "", p, flags=re.IGNORECASE).strip()

    # Se sobrar algo válido, incluir na lista final
    if p:
        produtos_limpos.append(p)

produtos_limpos

['Patrocinado\nTempero Granulado para Frango Maggi Meu Segredo Pacote 50g 10 Unidades de 5g Cada\nR$ 4,99\n-34%\nR$ 3,29\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Azeite Português Extra Virgem Andorinha Dorinha 500ml\nR$ 34,69\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Azeite Português Extra Virgem Tradicional Gallo 500ml\nR$ 38,89\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Azeite Terras de Camões Extra Virgem Frutado 500ml\nR$ 33,99\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Azeite Extra Virgem Allegro 500ml\nR$ 36,99\n-10%\nR$ 33,29\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Kit Azeite Terras de Camões Extra Virgem Frutado 500ml 3 Unidades\nR$ 101,97\n\nHiper Maceió Jatiúca\nAvenida Comendador Gustavo Paiva\nMangabeiras, Maceió - AL',
 'Azeite Italiano Extra Vir

In [12]:
len(produtos_limpos)

2129

In [13]:
# Lista original
dados_brutos = produtos_limpos

data_de_hoje = data_de_hoje
dados_estruturados = []

for linha in produtos_limpos:
    try:
        partes = linha.split("\n\n")
        produto_bloco = partes[0]
        endereco_bloco = partes[1] if len(partes) > 1 else ""

        # Quebra o bloco do produto em linhas individuais
        linhas = produto_bloco.strip().split("\n")

        # REMOVE TRECHOS tipo "/kg", "/g", "/L" apenas de dentro das linhas
        linhas_limpa = []
        for l in linhas:
            # Remove apenas os fragmentos como "R$ XX,XX/kg", "R$ XX,XX/L", etc
            l_limpo = re.sub(r"R\$ ?\d+,\d{2}/(kg|g|l|ml|100ml|un|litro)", "", l, flags=re.IGNORECASE)
            linhas_limpa.append(l_limpo.strip())

        # Nome do produto
        produto_nome = next((x for x in linhas_limpa if "R$" not in x and "%" not in x and "promoção" not in x.lower()), "").strip()

        # Preço final (último R$ válido depois da limpeza)
        precos_encontrados = [x for x in linhas_limpa if re.search(r"R\$ ?\d+,\d{2}", x)]
        preco_final = precos_encontrados[-1] if precos_encontrados else None
        if preco_final:
            preco_final = re.search(r"R\$ ?(\d+,\d{2})", preco_final).group(1)
            preco_final = float(preco_final.replace(",", "."))

        # Desconto
        desconto_linha = next((x for x in linhas_limpa if "%" in x and "-" in x), None)
        desconto = float(desconto_linha.replace("%", "").replace(",", ".").strip()) if desconto_linha else None

        # Loja e endereço
        endereco_linhas = endereco_bloco.strip().split("\n")
        loja = endereco_linhas[0] if len(endereco_linhas) > 0 else ""
        endereco = " - ".join(endereco_linhas[1:]) if len(endereco_linhas) > 1 else ""

        # Cidade e estado
        cidade_estado_texto = endereco_linhas[-1] if len(endereco_linhas) >= 1 else ""
        if "," in cidade_estado_texto and " - " in cidade_estado_texto:
            cidade_texto = cidade_estado_texto.split(",")[-1].strip()
            cidade, estado = cidade_texto.split(" - ")
        else:
            cidade, estado = "", ""

        # Monta o dicionário final
        dados_estruturados.append({
            "produto": produto_nome,
            "preco": preco_final,
            "desconto": desconto,
            "loja": loja.strip(),
            "endereco": endereco_bloco.strip(),
            "cidade": cidade.strip(),
            "estado": estado.strip(),
            "data": data_de_hoje
        })

    except Exception as e:
        print(f"[ERRO] Linha com problema:\n{linha}\n-> {e}")

# DataFrame final
df_produtos = pd.DataFrame(dados_estruturados)

# Garante que a coluna 'data' exista antes de reorganizar
if 'data' not in df_produtos.columns:
    df_produtos['data'] = data_de_hoje

# Reorganizar com 'data' na frente
colunas = ['data'] + [col for col in df_produtos.columns if col != 'data']
df_produtos = df_produtos[colunas]

#Incluir coluna de regioes
regioes_brasil = {
    'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste',
    'PE': 'Nordeste', 'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
}
df_produtos['regiao'] = df_produtos['estado'].map(regioes_brasil).fillna('Desconhecida')

# Alterar os dados da coluna cidade
df_produtos['cidade'] = df_produtos['cidade']+'/'+df_produtos['estado']

df_produtos.rename(columns={'estado': 'uf'}, inplace=True)

df_produtos

,data,produto,preco,desconto,loja,endereco,cidade,uf,regiao
0,2025-07-07,Patrocinado,3.29,-34.0,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
1,2025-07-07,Azeite Português Extra Virgem Andorinha Dorinh...,34.69,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
2,2025-07-07,Azeite Português Extra Virgem Tradicional Gall...,38.89,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
3,2025-07-07,Azeite Terras de Camões Extra Virgem Frutado 5...,33.99,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
4,2025-07-07,Azeite Extra Virgem Allegro 500ml,33.29,-10.0,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
...,...,...,...,...,...,...,...,...,...
2124,2025-07-07,Kit Azeite Português Extra Virgem Tradicional ...,104.37,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste
2125,2025-07-07,Azeite Allegro Extravirgem 250 ml,18.98,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste
2126,2025-07-07,Azeite Português Extra Virgem Andorinha Seleçã...,42.79,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste
2127,2025-07-07,Azeite Italiano Extra Virgem Tradicional Olitá...,43.99,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste


In [14]:
df_produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      2129 non-null   object 
 1   produto   2129 non-null   object 
 2   preco     2129 non-null   float64
 3   desconto  86 non-null     float64
 4   loja      2129 non-null   object 
 5   endereco  2129 non-null   object 
 6   cidade    2129 non-null   object 
 7   uf        2129 non-null   object 
 8   regiao    2129 non-null   object 
dtypes: float64(2), object(7)
memory usage: 149.8+ KB


In [15]:
df_produtos.to_csv('precos_carrefour_azeite_20250707.csv', index=False)

In [1]:
import pandas as pd

In [2]:
df_produtos = pd.read_csv('precos_carrefour_salmao_20250624.csv')

In [16]:
df_produtos.head(5)

,data,produto,preco,desconto,loja,endereco,cidade,uf,regiao
0,2025-07-07,Patrocinado,3.29,-34.0,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
1,2025-07-07,Azeite Português Extra Virgem Andorinha Dorinh...,34.69,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
2,2025-07-07,Azeite Português Extra Virgem Tradicional Gall...,38.89,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
3,2025-07-07,Azeite Terras de Camões Extra Virgem Frutado 5...,33.99,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
4,2025-07-07,Azeite Extra Virgem Allegro 500ml,33.29,-10.0,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste


In [17]:
df_produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      2129 non-null   object 
 1   produto   2129 non-null   object 
 2   preco     2129 non-null   float64
 3   desconto  86 non-null     float64
 4   loja      2129 non-null   object 
 5   endereco  2129 non-null   object 
 6   cidade    2129 non-null   object 
 7   uf        2129 non-null   object 
 8   regiao    2129 non-null   object 
dtypes: float64(2), object(7)
memory usage: 149.8+ KB


In [18]:
df_produtos.loja.unique()

array(['Hiper Maceió Jatiúca', 'Hiper Manaus Manpar',
       'Hiper Manaus Ponta Negra', 'Hiper Manaus Adrianópolis',
       'Hiper Manaus Shopping', 'Hiper Itabuna', 'Hiper Brasília Sul',
       'Hiper Águas Claras', 'Bairro Asa Norte Ii', 'Bairro Bom Motivo',
       'Hiper Asa Norte', 'Hiper Vila Velha', 'Hiper Goiânia Sul',
       'Hiper Avenida T9', 'Hiper Anápolis', 'Hiper Contagem',
       'Hiper BH Shopping', 'Hiper Pampulha', 'Hiper BH América',
       'Hiper Juiz de Fora', 'Hiper Uberlândia', 'Hiper Campo Grande',
       'Hiper Campina Grande', 'Hiper João Pessoa',
       'Hiper João Pessoa Bancários', 'Hiper Shopping Guararapes',
       'Hiper Shopping Petrolina', 'Hiper Avenida Recife',
       'Hiper Shopping Tacaruna', 'Hiper Recife Torre',
       'Hiper Recife Domingos Ferreira', 'Hiper Pinhais',
       'Hiper Champagnat', 'Hiper Parolim', 'Hiper Portão',
       'Hiper Cabral', 'Hiper Londrina', 'Hiper Campos de Goytacazes',
       'Hiper Duque de Caxias', 'Hiper Duque de 

In [19]:
df_lojas_geo = pd.read_csv('lista_lojas_carrefour.csv', sep=';')
df_lojas_geo['loja'].unique()

array(['Hiper Maceió Jatiúca', 'Hiper Manaus Manpar',
       'Hiper Manaus Ponta Negra', 'Hiper Manaus Adrianópolis',
       'Hiper Manaus Shopping', 'Hiper Itabuna', 'Hiper Brasília Sul',
       'Hiper Águas Claras', 'Bairro Asa Norte Ii', 'Bairro Bom Motivo',
       'Hiper Asa Norte', 'Hiper Vila Velha', 'Hiper Goiânia Sul',
       'Hiper Avenida T9', 'Hiper Anápolis', 'Hiper Contagem',
       'Hiper BH Shopping', 'Hiper Pampulha', 'Hiper BH América',
       'Hiper Juiz de Fora', 'Hiper Uberlândia', 'Hiper Campo Grande',
       'Hiper Campina Grande', 'Hiper João Pessoa',
       'Hiper João Pessoa Bancários', 'Hiper Shopping Guararapes',
       'Hiper Shopping Petrolina', 'Hiper Avenida Recife',
       'Hiper Shopping Tacaruna', 'Hiper Recife Torre',
       'Hiper Recife Domingos Ferreira', 'Hiper Pinhais',
       'Hiper Champagnat', 'Hiper Parolim', 'Hiper Portão',
       'Hiper Cabral', 'Hiper Londrina', 'Hiper Campos de Goytacazes',
       'Hiper Duque de Caxias', 'Hiper Duque de 

In [20]:
# Comparar lojas presentes em df_produtos mas não em df_lojas_geo
lojas_produtos = set(df_produtos.loja.unique())
lojas_geo = set(df_lojas_geo['loja'].unique())

lojas_somente_produtos = lojas_produtos - lojas_geo
lojas_somente_geo = lojas_geo - lojas_produtos

print("Lojas em df_produtos e não em df_lojas_geo:", lojas_somente_produtos)
print("Lojas em df_lojas_geo e não em df_produtos:", lojas_somente_geo)

Lojas em df_produtos e não em df_lojas_geo: {'Hiper Novo Hamburgo', 'Bairro Caçapava', 'Hiper Tietê'}
Lojas em df_lojas_geo e não em df_produtos: {'Hiper Taubaté Charles', 'Hiper Vila Maria', 'Bairro Vinhedo', 'Bairro Taubaté'}


In [21]:
# Nomes de lojas duplicadas
df_lojas_geo['loja'].value_counts().loc[lambda x: x > 1]


loja
Hiper Guarulhos       3
Hiper Osasco          3
Hiper Santa Maria     2
Hiper Campo Grande    2
Name: count, dtype: int64

In [2]:
import pandas as pd

In [4]:
df_produtos = pd.read_csv('carrefour_coleta_bacalhau_total.csv', sep=',')
df_produtos

,data,produto,preco,desconto,loja,endereco,cidade,uf,regiao
0,2025-07-12,Bacalhau Morhua Congelado em Postas Brasmar 800g,56.90,-20.0,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
1,2025-07-12,Lasca Salgada de Bacalhau Bacalanor 500g,26.89,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
2,2025-07-12,Lombo de Bacalhau sem Espinha Swift 1Kg,136.50,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
3,2025-07-12,Bacalhau Desfiado Congelado Brasmar 450g,42.89,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
4,2025-07-12,Posta De Bacalhau Dessalgado Riberalves 800 g,79.90,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste
...,...,...,...,...,...,...,...,...,...
803,2025-07-12,Bacalhau Desfiado Dessalgado Swift 500 g,40.90,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste
804,2025-07-12,Escondidinho de Bacalhau Bom Porto 400 g,33.79,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste
805,2025-07-12,Bacalhau Gadus Morhua Salgado Posta Resfriado ...,217.71,-10.0,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste
806,2025-07-12,Filé de Bacalhau Salgado Macrocephalus Carrefo...,179.99,-10.0,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste


In [5]:
df_lojas_geo = pd.read_csv('lista_lojas_carrefour.csv', sep=';')

# Mesclar as informações de geolocalização
df_geo_merge = pd.merge(df_produtos, df_lojas_geo[['loja', 'lat', 'long']],
                        on='loja', how='left')

df_geo_merge['lat'] = df_geo_merge['lat'].astype(str).str.replace(',', '.').astype(float)
df_geo_merge['long'] = df_geo_merge['long'].astype(str).str.replace(',', '.').astype(float)

# Identificar lojas sem geolocalização
lojas_sem_geo = df_geo_merge[df_geo_merge['lat'].isna()]['loja'].unique()

# Remover linhas com qualquer NaN
df_geo_merge = df_geo_merge[df_geo_merge['lat'].notna()]
# Remover linhas duplicadas (com base em todas as colunas)
df_geo_merge = df_geo_merge.drop_duplicates()

lojas_sem_geo.tolist()

['Hiper Novo Hamburgo', 'Bairro Caçapava', 'Hiper Tietê']

In [6]:
# Mesclar as informações de geolocalização
df_geo_merge = pd.merge(df_produtos, df_lojas_geo[['loja', 'lat', 'long']],
                        on='loja', how='left')

In [7]:
df_produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808 entries, 0 to 807
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      808 non-null    object 
 1   produto   808 non-null    object 
 2   preco     808 non-null    float64
 3   desconto  176 non-null    float64
 4   loja      808 non-null    object 
 5   endereco  808 non-null    object 
 6   cidade    808 non-null    object 
 7   uf        808 non-null    object 
 8   regiao    808 non-null    object 
dtypes: float64(2), object(7)
memory usage: 56.9+ KB


In [8]:
# Remover linhas duplicadas (com base em todas as colunas)
df_geo_merge = df_geo_merge.drop_duplicates()

In [9]:
df_geo_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 886 entries, 0 to 885
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   data      886 non-null    object 
 1   produto   886 non-null    object 
 2   preco     886 non-null    float64
 3   desconto  195 non-null    float64
 4   loja      886 non-null    object 
 5   endereco  886 non-null    object 
 6   cidade    886 non-null    object 
 7   uf        886 non-null    object 
 8   regiao    886 non-null    object 
 9   lat       859 non-null    object 
 10  long      859 non-null    object 
dtypes: float64(2), object(9)
memory usage: 76.3+ KB


In [10]:
df_geo_merge

,data,produto,preco,desconto,loja,endereco,cidade,uf,regiao,lat,long
0,2025-07-12,Bacalhau Morhua Congelado em Postas Brasmar 800g,56.90,-20.0,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,"-9,64959","-35,71739"
1,2025-07-12,Lasca Salgada de Bacalhau Bacalanor 500g,26.89,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,"-9,64959","-35,71739"
2,2025-07-12,Lombo de Bacalhau sem Espinha Swift 1Kg,136.50,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,"-9,64959","-35,71739"
3,2025-07-12,Bacalhau Desfiado Congelado Brasmar 450g,42.89,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,"-9,64959","-35,71739"
4,2025-07-12,Posta De Bacalhau Dessalgado Riberalves 800 g,79.90,NaN,Hiper Maceió Jatiúca,Hiper Maceió Jatiúca\nAvenida Comendador Gusta...,Maceió/AL,AL,Nordeste,"-9,64959","-35,71739"
...,...,...,...,...,...,...,...,...,...,...,...
881,2025-07-12,Bacalhau Desfiado Dessalgado Swift 500 g,40.90,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste,"-23,48623","-47,47117"
882,2025-07-12,Escondidinho de Bacalhau Bom Porto 400 g,33.79,NaN,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste,"-23,48623","-47,47117"
883,2025-07-12,Bacalhau Gadus Morhua Salgado Posta Resfriado ...,217.71,-10.0,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste,"-23,48623","-47,47117"
884,2025-07-12,Filé de Bacalhau Salgado Macrocephalus Carrefo...,179.99,-10.0,Hiper Sorocaba Norte,Hiper Sorocaba Norte\nAvenida Brasil\nTerra Ve...,Sorocaba/SP,SP,Sudeste,"-23,48623","-47,47117"


In [11]:
df_geo_merge.to_csv('precos_carrefour_bacalhau_20250712_loc.csv', index=False)